In [1]:
import pika
import numpy as np
import json
from sklearn.datasets import load_diabetes

In [4]:
X, y = load_diabetes(return_X_y=True)
random_row = np.random.randint(0, X.shape[0]-1)

In [5]:
random_row 

265

In [6]:
X[random_row]

array([-0.03457486,  0.05068012, -0.02560657, -0.01714685,  0.00118295,
       -0.00287962,  0.00814208, -0.01550765,  0.01482271,  0.04034337])

In [7]:
y[random_row]

74.0

In [8]:
# Подключение к серверу на локальном хосте:
connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
channel = connection.channel()

In [9]:
# Создадим очередь, с которой будем работать:
channel.queue_declare(queue='y_true')

<METHOD(['channel_number=1', 'frame_type=1', "method=<Queue.DeclareOk(['consumer_count=0', 'message_count=2', 'queue=y_true'])>"])>

In [10]:
# Опубликуем сообщение
# exchange определяет, в какую очередь отправляется сообщение. Если мы используем дефолтную точку обмена, то значение можно оставить пустым.
# параметр routing_key указывает имя очереди, 
# параметр body тело самого сообщения, 
channel.basic_publish(exchange='',
                      routing_key='y_true',
                      body=json.dumps(y[random_row]))


print('Сообщение с правильным ответом отправлено в очередь')

Сообщение с правильным ответом отправлено в очередь


In [11]:
channel.queue_declare(queue='Features')

<METHOD(['channel_number=1', 'frame_type=1', "method=<Queue.DeclareOk(['consumer_count=0', 'message_count=1', 'queue=Features'])>"])>

In [12]:
channel.basic_publish(exchange='',
                      routing_key='Features',
                      body=json.dumps(list(X[random_row])))

print('Сообщение с признаками отправлено в очередь')

Сообщение с признаками отправлено в очередь


In [13]:
# Закроем подключение 
connection.close()

In [14]:
json.dumps(y[random_row])

'74.0'

In [15]:
json.dumps(list(X[random_row]))

'[-0.034574862586967, 0.0506801187398187, -0.0256065714656645, -0.0171468461892456, 0.00118294589619092, -0.00287961973516629, 0.0081420836051921, -0.015507654304751, 0.0148227108412663, 0.0403433716478807]'